In [1]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.2 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.


In [2]:

from roboflow import Roboflow
rf = Roboflow(api_key="sjY1k7Bc5AwCbbatoIz2")
project = rf.workspace("lebeling-sendiri-300").project("-25m-3class")
version = project.version(4)
dataset = version.download("tfrecord")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to @25M(3class)-4 in tfrecord:: 100%|██████████| 11/11 [00:00<00:00, 71.99it/s]


In [3]:
# Clone the tensorflow models repository from GitHub
!pip uninstall Cython -y # Temporary fix for "No module named 'object_detection'" error
!git clone --depth 1 https://github.com/tensorflow/models

Found existing installation: Cython 3.0.11
Uninstalling Cython-3.0.11:
  Successfully uninstalled Cython-3.0.11
Cloning into 'models'...
remote: Enumerating objects: 4352, done.
remote: Counting objects: 100% (4352/4352), done.
remote: Compressing objects: 100% (3160/3160), done.
remote: Total 4352 (delta 1188), reused 3997 (delta 1119), pack-reused 0 (from 0)
Receiving objects: 100% (4352/4352), 54.72 MiB | 36.99 MiB/s, done.
Resolving deltas: 100% (1188/1188), done.


In [4]:
# Copy setup files into models/research folder
!cd /kaggle/working/models/research && protoc object_detection/protos/*.proto --python_out=.
# !cd models/research/ && protoc object_detection/protos/*.proto --python_out=/kaggle/working/models/research/object_detection

#cp object_detection/packages/tf2/setup.py .


In [5]:
# Modify setup.py file to install the tf-models-official repository targeted at TF v2.8.0
import re
with open('/kaggle/working/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open('/kaggle/working/models/research/setup.py', 'w') as f:
    # Set fine_tune_checkpoint path
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.8.0', s)
    f.write(s)

In [6]:
# Install the Object Detection API (NOTE: This block takes about 10 minutes to finish executing)

# Need to do a temporary fix with PyYAML because Colab isn't able to install PyYAML v5.4.1
!pip install pyyaml==5.3

# Need to downgrade to TF v2.8.0 due to Colab compatibility bug with TF v2.10 (as of 10/03/22)
!pip install tensorflow==2.8.0
# !pip install /kaggle/working/models/research

# Install CUDA version 11.0 (to maintain compatibility with TF v2.8.0)
!pip install tensorflow_io==0.23.1
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/11.0.2/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub
!apt-get update && sudo apt-get install -y cuda-toolkit-11-0  # Thêm cờ --yes vào đây
!export LD_LIBRARY_PATH=/usr/local/cuda-11.0/lib64:$LD_LIBRARY_PATH

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.2/268.2 kB 6.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.3-cp310-cp310-linux_x86_64.whl size=44244 sha256=f3b9096ac7cb4d6d7b05090708ca49b936a999ec68735da308c98ce7ee517d2d
  Stored in directory: /root/.cache/pip/wheels/0d/72/68/a263cfc14175636cf26bada99f13b735be1b60a11318e08bfc
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.2
    Uninstalling PyYAML-6.0.2:
      Successfully uninstalled PyYAML-6.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 44.0.1 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.0.0 which is incompatible.
dask 2024.12.1 requires pyyaml>=5.3.1, but you h

In [7]:
# Run Model Bulider Test file, just to verify everything's working properly
!pip install /kaggle/working/models/research

!python /kaggle/working/models/research/object_detection/builders/model_builder_tf2_test.py

Processing ./models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 40.

In [8]:
# Change the chosen_model variable to deploy different models available in the TF2 object detection zoo
chosen_model = 'ssd-mobilenet-v2-fpnlite-320'

MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },
    # The centernet model isn't working as of 9/10/22
    #'centernet-mobilenet-v2': {
    #    'model_name': 'centernet_mobilenetv2fpn_512x512_coco17_od',
    #    'base_pipeline_file': 'pipeline.config',
    #    'pretrained_checkpoint': 'centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz',
    #}
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [9]:

# Create "mymodel" folder for holding pre-trained weights and configuration files
%mkdir /kaggle/working/models/mymodel/
%cd /kaggle/working/models/mymodel/

# Download pre-trained model weights
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download training configuration file for model
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

/kaggle/working/models/mymodel
--2025-06-20 07:06:17--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.12.207, 74.125.26.207, 172.217.204.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.12.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  --.-KB/s    in 0.1s    

2025-06-20 07:06:17 (165 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

--2025-06-20 07:06:18--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133,

In [10]:
# Set training parameters for the model
num_steps =1000
batch_size = 16

# print(batch_size)

In [11]:
import os

def get_file_by_extension(folder, extension):
    """Mencari file dengan ekstensi tertentu di dalam folder."""
    for file in os.listdir(folder):
        if file.endswith(extension):
            return os.path.join(folder, file)
    return None  # Jika tidak ditemukan

# Dataset di Kaggle biasanya ada di folder 
dataset_location = dataset.location

# Folder train dan test
train_folder = os.path.join(dataset_location, 'train')
val_folder = os.path.join(dataset_location, 'valid')
test_folder = os.path.join(dataset_location, 'test')

# Cari file .tfrecord dan .pbtxt secara otomatis
test_record_fname = get_file_by_extension(test_folder, ".tfrecord")
val_record_fname = get_file_by_extension(val_folder, ".tfrecord")
train_record_fname = get_file_by_extension(train_folder, ".tfrecord")
label_map_pbtxt_fname = get_file_by_extension(train_folder, ".pbtxt")
# Set file locations and get number of classes for config file
pipeline_fname = '/kaggle/working/models/mymodel/' + base_pipeline_file
fine_tune_checkpoint = '/kaggle/working/models/mymodel/' + model_name + '/checkpoint/ckpt-0'
# Cetak hasil
print("Test Record:", test_record_fname)
print("Val Record:", val_record_fname)
print("Train Record:", train_record_fname)
print("Label Map:", label_map_pbtxt_fname)

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)
print('Total classes:', num_classes)


Test Record: /kaggle/working/@25M(3class)-4/test/1-QDa5-FSKD.tfrecord
Val Record: /kaggle/working/@25M(3class)-4/valid/1-QDa5-FSKD.tfrecord
Train Record: /kaggle/working/@25M(3class)-4/train/1-QDa5-FSKD.tfrecord
Label Map: /kaggle/working/@25M(3class)-4/train/1-QDa5-FSKD_label_map.pbtxt
Total classes: 3


In [12]:
# Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file
import re

#train_record_fname = '/kaggle/working/vehicle-detection-1/train/vehicles.tfrecord'
#val_record_fname = '/kaggle/working/vehicle-detection-1/valid/vehicles.tfrecord'

%cd /kaggle/working/models/mymodel
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(16), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # If using ssd-mobilenet-v2, reduce learning rate (because it's too high in the default config file)
    if chosen_model == 'ssd-mobilenet-v2-fpnlite-320':
      s = re.sub('learning_rate_base: 0.001',
                 'learning_rate_base: 0.0001', s)

      s = re.sub('warmup_learning_rate: 0.001',
                 'warmup_learning_rate: 0.001', s)

    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'ssd-mobilenet-v2-fpnlite-320':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)
   

    f.write(s)



/kaggle/working/models/mymodel
writing custom configuration file


In [13]:
# Set the path to the custom config file and the directory to store training checkpoints in
# %mkdir /content/training
pipeline_file = '/kaggle/working/models/mymodel/pipeline_file.config'
model_dir = '/kaggle/working/training/'

In [14]:

%%writefile /kaggle/working/models/mymodel/pipeline_file.config
# SSD with Mobilenet v2 FPN-lite (go/fpn-lite) feature extractor, shared box
# predictor and focal loss (a mobile version of Retinanet).
# Retinanet: see Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 22.2 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 3
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 1.0
        aspect_ratios: [0.5, 0.75, 1.0, 1.5]
        scales_per_octave: 2
      }
    }
    image_resizer {
      fixed_shape_resizer {
        height: 512
        width: 512
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        depth: 128
        class_prediction_bias_init: -4.6
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
              weight: 0.00004
            }
          }
          initializer {
            random_normal_initializer {
              stddev: 0.01
              mean: 0.0
            }
          }
          batch_norm {
            scale: true,
            decay: 0.997,
            epsilon: 0.001,
          }
        }
        num_layers_before_predictor: 4
        share_prediction_tower: true
        use_depthwise: true
        kernel_size: 3
      }
    }
    feature_extractor {
      type: 'ssd_mobilenet_v2_fpn_keras'
      use_depthwise: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.00004
          }
        }
        initializer {
          random_normal_initializer {
            stddev: 0.01
            mean: 0.0
          }
        }
        batch_norm {
          scale: true,
          decay: 0.997,
          epsilon: 0.001,
        }
      }
      override_base_feature_extractor_hyperparams: true
    }
    loss {
      classification_loss {
        weighted_sigmoid_focal {
          alpha: 0.35
          gamma: 2.0
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_weight: 2.0
      localization_weight: 2.0
    }
    normalize_loss_by_num_matches: true
    normalize_loc_loss_by_codesize: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
  }
}

train_config {
  fine_tune_checkpoint_version: V2
  fine_tune_checkpoint: "/kaggle/working/models/mymodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0"
  fine_tune_checkpoint_type: "detection"
  batch_size: 16
  sync_replicas: true
  startup_delay_steps: 0
  replicas_to_aggregate: 8
  num_steps: 80000
  


  data_augmentation_options {
    random_horizontal_flip {}
  }
  data_augmentation_options {
    random_crop_image {
      min_object_covered: 0.0
      min_aspect_ratio: 0.5
      max_aspect_ratio: 1.7
      min_area: 0.75
      max_area: 1.0
      overlap_thresh: 0.0
    }
  }

  data_augmentation_options {
    random_adjust_hue {
    }
  }

  data_augmentation_options {
    random_adjust_contrast {
    }
  }

  data_augmentation_options {
    random_adjust_saturation {
    }
  }

  data_augmentation_options {
    random_adjust_brightness {
    }
  }

  data_augmentation_options {
   random_vertical_flip {
   }
 }
  data_augmentation_options {
   random_rotation90 {
   }
 }

  optimizer {
    adam_optimizer {
      learning_rate {
        cosine_decay_learning_rate {
          learning_rate_base: 0.00002
          total_steps: 80000
          warmup_learning_rate: 0.00002
          warmup_steps: 1000
        }
      }
     
      epsilon: 1e-8
    }
    use_moving_average: false
  }

  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
}


 

train_input_reader: {
  label_map_path: "/kaggle/working/@25M(3class)-4/train/1-QDa5-FSKD_label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/kaggle/working/@25M(3class)-4/train/1-QDa5-FSKD.tfrecord"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
}

eval_input_reader: {
  label_map_path: "/kaggle/working/@25M(3class)-4/train/1-QDa5-FSKD_label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/kaggle/working/@25M(3class)-4/valid/1-QDa5-FSKD.tfrecord"
  }
}

Overwriting /kaggle/working/models/mymodel/pipeline_file.config


In [15]:
# (Optional) Display the custom configuration file's contents
!cat /kaggle/working/models/mymodel/pipeline_file.config

# SSD with Mobilenet v2 FPN-lite (go/fpn-lite) feature extractor, shared box
# predictor and focal loss (a mobile version of Retinanet).
# Retinanet: see Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 22.2 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 3
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      multiscale_anchor_generator {

In [16]:
 #%load_ext tensorboard
 #%reload_ext tensorboard
 #%tensorboard --logdir '/kaggle/working/training/train' --port 6009



In [17]:
# # Run training!
# !python /kaggle/working/models/research/object_detection/model_main_tf2.py \
#     --pipeline_config_path={pipeline_file} \
#     --model_dir={model_dir} \
#     --alsologtostderr \
#     --num_train_steps={num_steps} \
#     --sample_1_of_n_eval_examples=1

pipeline_file = "/kaggle/working/models/mymodel/pipeline_file.config"
model_dir = "/kaggle/working/training"
num_iterations = 4          # berapa kali train+eval akan diulang
steps_per_iter = 12500        # jumlah step setiap loop

for i in range(num_iterations):
    print(f"\n🟢 Training iterasi ke-{i+1} ({(i)*steps_per_iter} -> {(i+1)*steps_per_iter} steps)")
    !python /kaggle/working/models/research/object_detection/model_main_tf2.py \
        --pipeline_config_path={pipeline_file} \
        --model_dir={model_dir} \
        --alsologtostderr \
        --num_train_steps={(i+1)*steps_per_iter}

    print(f"\n🔵 Evaluasi setelah iterasi ke-{i+1}")
    !python /kaggle/working/models/research/object_detection/model_main_tf2.py \
        --pipeline_config_path={pipeline_file} \
        --model_dir={model_dir} \
        --checkpoint_dir={model_dir} \
        --alsologtostderr \
        --sample_1_of_n_eval_examples=1





🟢 Training iterasi ke-1 (0 -> 12500 steps)
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file 

In [27]:
# # Make a directory to store the trained TFLite model
# !mkdir /kaggle/working/custom_model_lite
# output_directory = '/kaggle/working/custom_model_lite'

# # Path to training directory (the conversion script automatically chooses the highest checkpoint file)
# last_model_path = '/kaggle/working/training'

# !python /kaggle/working/models/research/object_detection/export_tflite_graph_tf2.py \
#     --trained_checkpoint_dir {last_model_path} \
#     --output_directory {output_directory} \
#     --pipeline_config_path {pipeline_file}


import os
import subprocess

# Path ke file pipeline config (hasil modifikasi mu)
pipeline_config_path = '/kaggle/working/models/mymodel/pipeline_file.config'

# Folder checkpoint hasil training
trained_checkpoint_dir = '/kaggle/working/training/'

# Folder output untuk saved_model
output_directory = '/kaggle/working/training/saved_model'

# Pastikan folder output sudah ada
os.makedirs(output_directory, exist_ok=True)

# Jalankan exporter_main_v2.py dari TensorFlow Object Detection API
cmd = [
    'python',
    '/kaggle/working/models/research/object_detection/exporter_main_v2.py',
    '--input_type', 'image_tensor',
    '--pipeline_config_path', pipeline_config_path,
    '--trained_checkpoint_dir', trained_checkpoint_dir,
    '--output_directory', output_directory
]

print("Menjalankan export saved_model ...")
subprocess.run(cmd, check=True)
print(f"Saved_model berhasil diekspor ke: {output_directory}")


Menjalankan export saved_model ...
Saved_model berhasil diekspor ke: /kaggle/working/training/saved_model


In [19]:
!zip -r /kaggle/working/training.zip /kaggle/working/training/saved_model

  adding: kaggle/working/training/saved_model/ (stored 0%)
  adding: kaggle/working/training/saved_model/pipeline.config (deflated 70%)
  adding: kaggle/working/training/saved_model/checkpoint/ (stored 0%)
  adding: kaggle/working/training/saved_model/checkpoint/ckpt-0.data-00000-of-00001 (deflated 8%)
  adding: kaggle/working/training/saved_model/checkpoint/ckpt-0.index (deflated 80%)
  adding: kaggle/working/training/saved_model/checkpoint/checkpoint (deflated 41%)
  adding: kaggle/working/training/saved_model/saved_model/ (stored 0%)
  adding: kaggle/working/training/saved_model/saved_model/assets/ (stored 0%)
  adding: kaggle/working/training/saved_model/saved_model/saved_model.pb (deflated 92%)
  adding: kaggle/working/training/saved_model/saved_model/variables/ (stored 0%)
  adding: kaggle/working/training/saved_model/saved_model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: kaggle/working/training/saved_model/saved_model/variables/variables.index (deflated 78%)

In [28]:
!zip -r /kaggle/working/training.zip /kaggle/working/training

updating: kaggle/working/training/saved_model/ (stored 0%)
updating: kaggle/working/training/saved_model/pipeline.config (deflated 70%)
updating: kaggle/working/training/saved_model/checkpoint/ (stored 0%)
updating: kaggle/working/training/saved_model/checkpoint/ckpt-0.data-00000-of-00001 (deflated 8%)
updating: kaggle/working/training/saved_model/checkpoint/ckpt-0.index (deflated 80%)
updating: kaggle/working/training/saved_model/checkpoint/checkpoint (deflated 40%)
updating: kaggle/working/training/saved_model/saved_model/ (stored 0%)
updating: kaggle/working/training/saved_model/saved_model/assets/ (stored 0%)
updating: kaggle/working/training/saved_model/saved_model/saved_model.pb (deflated 92%)
updating: kaggle/working/training/saved_model/saved_model/variables/ (stored 0%)
updating: kaggle/working/training/saved_model/saved_model/variables/variables.data-00000-of-00001 (deflated 8%)
updating: kaggle/working/training/saved_model/saved_model/variables/variables.index (deflated 78%)

In [21]:
import shutil
import os
from zipfile import ZipFile
from IPython.display import FileLink

# Nama file output zip
zip_file = "/kaggle/working/backup.zip"

# Daftar folder yang ingin dikompres
folders_to_zip = ["training"]


# Buat file ZIP dan tambahkan folder yang dipilih
with ZipFile(zip_file, 'w') as zipf:
    for folder in folders_to_zip:
        folder_path = os.path.join("/kaggle/working", folder)
        if os.path.exists(folder_path):
            for root, _, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, os.path.relpath(file_path, "/kaggle/working"))

# Tampilkan link untuk mengunduh file zip
if os.path.exists(zip_file):
    display(FileLink(zip_file))
else:
    print("File ZIP tidak ditemukan.")


/kaggle/working/backup.zip

In [22]:
# import shutil

# zip_path = '/kaggle/working/saved_model.zip'
# saved_model_dir = '/kaggle/working/training/saved_model'

# # Kompres ke zip
# shutil.make_archive(base_name=zip_path.replace('.zip', ''), format='zip', root_dir=saved_model_dir)
# shutil.make_archive('/kaggle/working/training_full_backup', 'zip', '/kaggle/working/training')

# print(f"✅ Model berhasil dikompres menjadi: {zip_path}")


In [23]:
# # Convert exported graph file into TFLite model file
# import tensorflow as tf

# converter = tf.lite.TFLiteConverter.from_saved_model('/kaggle/working/custom_model_lite/saved_model')
# tflite_model = converter.convert()

# with open('/kaggle/working/custom_model_lite/detect.tflite', 'wb') as f:
#   f.write(tflite_model)

In [24]:
# # Run evaluation!
# !python /kaggle/working/models/research/object_detection/model_main_tf2.py \
#     --pipeline_config_path={pipeline_file} \
#     --model_dir={model_dir} \
#     --checkpoint_dir={model_dir} \
#     --alsologtostderr \
#     --sample_1_of_n_eval_examples=1


# !python /kaggle/working/models/research/object_detection/model_main_tf2.py \
#     --pipeline_config_path={pipeline_file} \
#     --model_dir={model_dir} \
#     --checkpoint_dir={model_dir} \
#     --alsologtostderr \
#     --sample_1_of_n_eval_examples=1



In [25]:
# # Buat zip dari direktori log
# !zip -r model_logs.zip /kaggle/working/training


In [26]:
# !rm /kaggle/working/custom_model_lite/detect.tflite